# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into Deep Neural Networks for NLP!

## Objectives

You will be able to:

* Demonstrate a basic understanding of the architecture of the Word2Vec model
* Demonstrate an understanding of the various tunable parameters of Word2Vec such as vector size and window size

## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec Model. Then, you'll move onto building Neural Networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a github repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json` function from pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've loaded in the data, inspect the head of the DataFrame to see what your data looks like. 

In [3]:
raw_df = pd.read_json('News_Category_Dataset_v2.zip', lines = True)
raw_df

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26
...,...,...,...,...,...,...
200848,TECH,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,"Reuters, Reuters",https://www.huffingtonpost.com/entry/rim-ceo-t...,Verizon Wireless and AT&T are already promotin...,2012-01-28
200849,SPORTS,Maria Sharapova Stunned By Victoria Azarenka I...,,https://www.huffingtonpost.com/entry/maria-sha...,"Afterward, Azarenka, more effusive with the pr...",2012-01-28
200850,SPORTS,"Giants Over Patriots, Jets Over Colts Among M...",,https://www.huffingtonpost.com/entry/super-bow...,"Leading up to Super Bowl XLVI, the most talked...",2012-01-28
200851,SPORTS,Aldon Smith Arrested: 49ers Linebacker Busted ...,,https://www.huffingtonpost.com/entry/aldon-smi...,CORRECTION: An earlier version of this story i...,2012-01-28


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data--`headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `combined_text` that consists of the data from `df.headline` plus a space character (`' '`) plus the data from `df.short_description`.
* Use the `combined_text` column's `map()` function and pass in `word_tokenize`. Store the result returned in `data`.

In [9]:
raw_df.headline[0]

'There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV'

In [10]:
raw_df.short_description[0]

'She left her husband. He killed their children. Just another day in America.'

In [6]:
[raw_df.headline[i] + ' ' + raw_df.short_description[i] for i in range(len(raw_df))]

['There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV She left her husband. He killed their children. Just another day in America.',
 "Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song Of course it has a song.",
 'Hugh Grant Marries For The First Time At Age 57 The actor and his longtime girlfriend Anna Eberstein tied the knot in a civil ceremony.',
 "Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork The actor gives Dems an ass-kicking for not fighting hard enough against Donald Trump.",
 'Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog The "Dietland" actress said using the bags is a "really cathartic, therapeutic moment."',
 'Morgan Freeman \'Devastated\' That Sexual Harassment Claims Could Undermine Legacy "It is not right to equate horrific incidents of sexual assault with misplaced compliments or humor," he said in a statement.',
 "Donald Trump Is Lovin' New McDonald's Jingle In 'Tonight Show' Bit It

In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/karolinajoz/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
raw_df['combined_text'] = [raw_df.headline[i] + ' ' + raw_df.short_description[i] for i in range(len(raw_df))]
data = raw_df.combined_text.map(lambda x: word_tokenize(x))

Inspect the first 5 items in `data` to see how everything looks. 

In [17]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from gensim below. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1`.
    * The number of threads to use during training, `workers=4`

In [18]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers = 4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call `model.train()` and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count`. 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [20]:
model.train(data, total_examples=model.corpus_count, epochs = 10)

(55561978, 67352790)

Great! you now have a fully trained model! The word vectors themselves are stored inside of a `Word2VecKeyedVectors` instance, which is stored inside of `model.wv`. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [21]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [31]:
wv.most_similar(['Texas'])

[('Ohio', 0.8315455317497253),
 ('Maryland', 0.8183360695838928),
 ('Pennsylvania', 0.8092271685600281),
 ('Oregon', 0.8036277890205383),
 ('Georgia', 0.8014289140701294),
 ('California', 0.7981974482536316),
 ('Florida', 0.7925111055374146),
 ('Louisiana', 0.7869457602500916),
 ('Wisconsin', 0.7838336825370789),
 ('Arizona', 0.7819352746009827)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `most_similar()` function's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [32]:
wv.most_similar(negative = ['Texas'])

[('ThankfulForMuslimWomen', 0.4160316288471222),
 ('winter-themed', 0.38815754652023315),
 ('boredom', 0.3861379027366638),
 ('CLOSErikers', 0.383649617433548),
 ('sparks', 0.3815665543079376),
 ('relegates', 0.38019615411758423),
 ('evokes', 0.3781243860721588),
 ('CheerioChallenge', 0.37522387504577637),
 ('Butchering', 0.37481796741485596),
 ('feeds', 0.37384292483329773)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [25]:
wv['Texas']

array([-2.9146595 ,  0.84070873, -0.07228981, -0.05209581, -0.9496216 ,
       -1.1494405 , -1.3689812 ,  2.0536082 ,  1.6057185 ,  2.809812  ,
        1.6768998 ,  0.7773849 , -0.03888997,  0.38386217, -0.08726957,
       -1.2774116 , -1.3804616 , -1.7618091 ,  1.4919287 ,  1.7500261 ,
        0.07479392,  0.9029037 , -0.1985072 , -0.5480803 ,  1.8880528 ,
       -2.7138846 ,  1.8556969 , -0.6293729 ,  1.0068007 ,  1.6899852 ,
        2.6446133 ,  1.8998617 ,  0.03834797, -0.6563153 , -0.29664975,
       -0.19027525,  0.57214373,  1.4222721 , -0.64060587, -0.45910078,
        1.5204575 , -1.713205  ,  0.37753487,  0.6747862 ,  1.2857738 ,
       -1.3618842 ,  1.9427354 ,  0.99081683, -0.64444005,  0.94487226,
       -3.2040756 , -1.3352773 ,  1.7527905 , -0.05607297, -0.53653073,
        0.42326522, -0.16163506,  1.7281619 , -2.5805955 , -0.16317628,
       -1.5833907 , -0.47720528, -0.04111491, -0.44161305, -0.14673856,
       -0.33976924, -0.1611682 ,  2.6523607 ,  0.36453506, -0.11

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [26]:
wv.vectors

array([[-2.0690796 ,  1.241539  ,  0.56006694, ...,  0.12662953,
        -1.280781  , -0.95214736],
       [-1.990862  ,  1.2059747 ,  1.2515937 , ...,  1.1688561 ,
        -3.7858303 , -0.74895394],
       [-0.7609686 ,  0.5012957 ,  1.4299554 , ...,  1.4481311 ,
        -1.150396  , -1.5251683 ],
       ...,
       [ 0.07541669, -0.00655367, -0.03360619, ..., -0.05249458,
        -0.06197269,  0.06504606],
       [-0.03755824, -0.03462103,  0.04059184, ...,  0.02841545,
        -0.02777577, -0.06005402],
       [-0.01263003,  0.02253408,  0.02551249, ..., -0.03175749,
        -0.08515415,  0.04242793]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `most_similar` function and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [30]:
wv.most_similar(positive = ['king', 'woman'], negative = ['man'])

[('villain', 0.6031278967857361),
 ('revival', 0.5993286967277527),
 ('princess', 0.5940976142883301),
 ('brunette', 0.5892631411552429),
 ('craze', 0.5825107097625732),
 ('queen', 0.5817798972129822),
 ('monster', 0.5650883316993713),
 ('title', 0.5643635392189026),
 ('du', 0.5610139966011047),
 ('crown', 0.5583409667015076)]

As you can see from the output above, your model isn't perfect, but 'Queen' is still in the top 3, and with 'Princess' not too far behind. As you can see from the word in first place, 'reminiscent', your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to created vectorized word embeddings!